<a href="https://colab.research.google.com/github/mmontes-umag/datos-intro-bioinf/blob/main/Pr%C3%A1ctico_Control_de_calidad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Control de calidad de datos de secuenciación <br>
En este notebook analizaremos lecturas brutas del patógeno Vibrio cholerae. La muestra fue aislada a partir de reservas de agua y el [proyecto de secuenciación](https://www.ebi.ac.uk/ena/browser/view/PRJNA510624) está disponible en la base de datos EMBL-EBI. <br>
Para este práctico usaremos las lecturas contenidas en el archivo [SRR8364450](https://www.ebi.ac.uk/ena/browser/view/SRR8364450), sin embargo no usaremos el archivo completo, en vez de ello trabajaremos con una sub-muestra de 10,000 lecturas del total de 1,343,093 lecturas disponibles en el archivo completo.<br>
Antes de comenzar puedes confirmar la tecnología de secuenciación utilizada por los autores del estudio, así como el formato de los datos siguiendo el link del archivo de lecturas.<br>
<br>
### Instalación de herramientas 
Las siguientes tres celdas de código instalarán el administrador de paquetes [Miniconda](https://docs.conda.io/en/latest/miniconda.html). 

In [7]:
# Correr esta celda instala el administrador de paquetes Miniconda. Usar una sola vez por sesión.
%%bash
MINICONDA_INSTALLER_SCRIPT=Miniconda3-4.5.12-Linux-x86_64.sh
MINICONDA_PREFIX=/usr/local
wget https://repo.continuum.io/miniconda/$MINICONDA_INSTALLER_SCRIPT
chmod +x $MINICONDA_INSTALLER_SCRIPT
./$MINICONDA_INSTALLER_SCRIPT -b -f -p $MINICONDA_PREFIX

PREFIX=/usr/local
reinstalling: python-3.7.1-h0371630_7 ...
Python 3.7.1
reinstalling: ca-certificates-2018.03.07-0 ...
reinstalling: conda-env-2.6.0-1 ...
reinstalling: libgcc-ng-8.2.0-hdf63c60_1 ...
reinstalling: libstdcxx-ng-8.2.0-hdf63c60_1 ...
reinstalling: libffi-3.2.1-hd88cf55_4 ...
reinstalling: ncurses-6.1-he6710b0_1 ...
reinstalling: openssl-1.1.1a-h7b6447c_0 ...
reinstalling: xz-5.2.4-h14c3975_4 ...
reinstalling: yaml-0.1.7-had09818_2 ...
reinstalling: zlib-1.2.11-h7b6447c_3 ...
reinstalling: libedit-3.1.20170329-h6b74fdf_2 ...
reinstalling: readline-7.0-h7b6447c_5 ...
reinstalling: tk-8.6.8-hbc83047_0 ...
reinstalling: sqlite-3.26.0-h7b6447c_0 ...
reinstalling: asn1crypto-0.24.0-py37_0 ...
reinstalling: certifi-2018.11.29-py37_0 ...
reinstalling: chardet-3.0.4-py37_1 ...
reinstalling: idna-2.8-py37_0 ...
reinstalling: pycosat-0.6.3-py37h14c3975_0 ...
reinstalling: pycparser-2.19-py37_0 ...
reinstalling: pysocks-1.6.8-py37_0 ...
reinstalling: ruamel_yaml-0.15.46-py37h14c3975

--2021-08-04 15:50:30--  https://repo.continuum.io/miniconda/Miniconda3-4.5.12-Linux-x86_64.sh
Resolving repo.continuum.io (repo.continuum.io)... 104.18.201.79, 104.18.200.79, 2606:4700::6812:c94f, ...
Connecting to repo.continuum.io (repo.continuum.io)|104.18.201.79|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://repo.anaconda.com/miniconda/Miniconda3-4.5.12-Linux-x86_64.sh [following]
--2021-08-04 15:50:30--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.12-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.131.3, 104.16.130.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.131.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69826864 (67M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.12-Linux-x86_64.sh’

     0K .......... .......... .......... .......... ..........  0% 7.36M 9s
    50K .......... .......... .......... .......... ..........

In [8]:
# Correr esta celda complementa la instalación de la celda anterior. Usar una sola vez por sesión.
import sys
sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython']

In [9]:
# Correr esta celda complementa la instalación de las dos celdas anteriores. Usar una sola vez por sesión.

_ = (sys.path
        .append("/usr/local/lib/python3.7/site-packages"))

La celda a continuación instala la herramienta vista en clases, [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/), usada para evaluar la calidad de datos de secuenciación.

In [11]:
# Correr esta celda instala la herramienta FastQC. Usar una sola vez por sesión.
! conda install -c bioconda fastqc --yes

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - 

La celda a continuación instala la herramienta [Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic), usada para cortar y filtrar lecturas, según su calidad.

In [12]:
# Correr esta celda instala la herramienta Trimmomatic. Usar una sola vez por sesión.
! conda install -c bioconda trimmomatic --yes

Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - trimmomatic


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    tqdm-4.62.0                |     pyhd3eb1b0_1          80 KB
    openssl-1.1.1k             |       h27cfd23_0         3.8 MB
    conda-4.10.3               |   py37h06a4308_0         3.1 MB
    certifi-2021.5.30          |   py37h06a4308_0         141 KB
    trimmomatic-0.39           |       hdfd78af_2         144 KB  bioconda
    conda-package-handling-1.7.3|   py37h27cfd23_1         962 KB
    --------------

### Descarga de datos

Las lecturas que analizaremos están disponibles en este [link](https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/sub1_10000.fastq).<br>
La siguiente celda descargará los datos en la nube de Colab.

In [15]:
! wget https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/sub1_10000.fastq

--2021-08-04 15:55:50--  https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/sub1_10000.fastq
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5323458 (5.1M) [text/plain]
Saving to: ‘sub1_10000.fastq’

sub1_10000.fastq    100%[===================>]   5.08M  22.2MB/s    in 0.2s    

2021-08-04 15:55:50 (22.2 MB/s) - ‘sub1_10000.fastq’ saved [5323458/5323458]



Podemos revisar las primeras tres lecturas del archivo que obtuvimos *sub1_10000.fastq*, incluyendo la información de calidad, en la celda a continuación.

In [16]:
! less sub1_10000.fastq | head -n 12

@SRR8364450.216106 216106/1
CTCTAGCGGAGTAACACCCGCCCAATCCACGCCCGGTTCTTCAGGAGCGTAGTCTTGAACGATATCGAGATAGGTAAATTGTCCCGGCTGCAAAATCACGTCAAAGCCTTGTTTGGCGCAATCCAAGGCGGCTTTTTCCGATAGCCAAGAGTAAATCACCGTATCTTTACTCACTTTGTCGCCGTGATGAGCTTCTTCCCAGCCGACCATACGCTTACCCAAGCTCTTGAGTTTTTTCTCGGCGTAGCGC
+
DDDDDFFDDCCDGGGGGGGGGGGGHHHHHHGGGGGGGGGHHHHHHHGHGGGGGHHHHHHHHGHGGHGHGHGHHHGHHHHHHHHHHGGGGGGHHHHHHHHGHHGGHHHHHHHHHHHGHGGGGGHHHHHGHGGGGGHHHHHGGGGGHHHHHHHGGHHHHHHHGGGGGGGGGGGGGGHGGGGGGGGGGGGFFFFFHFFFHHHFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFHFFFFFFFFFFFFFFF
@SRR8364450.576388 576388/1
CCCTCGATGCGATGCAAGTACGCGATGCGCTGGTCACCATCGAAGACAAAGGTGCGCTGGATTGTGTGATCCGCGCGCGCGTACAGGCGGCGGTGATGCGCGCTTGTGATGTTCAGAACATTGAGTGGAGCCAGCTATCATGACCAAACTTCGTCGCAGTATGTTATCTGTCTCTTATACACATCTCCGAGACCACGAGACCTCTCTACATCTCGTATTCCGGCTTCTGCTTGTGAAAAAAAATAGTTTA
+
ABAAA3AFFB@B4AEGG4GGGFFAA2AEFGECCHHFBGHHG?HBFGHCCBF2FFGEEEEE1GGFFFHHHHHHGGGGG>E@?ECGFFBC@CCG<><EHHFDDDC-CE:CFGHFCGFFC;CBBBFBG0;EBEBFBAFBFFFFGFFG/BAEGGFGFBABB<.9;9F/;BBFFF//BFFFFFFFFFFFFB9

Sin embargo, revisar las lecturas directamente como en la celda anterior no es muy informativo acerca de la calidad general de los datos. Para ello usaremos la herramienta FastQC.

### Evaluación de calidad con FastQC

FastQC es una aplicación java compatible con Windows, macOS, y Linux. Se puede usar como interfaz gráfica o mediante la línea de comandos. En este caso usaremos la segunda opción en la celda a continuación, pero siéntete libre de [descargar y probar FastQC](https://raw.githubusercontent.com/s-andrews/FastQC/master/INSTALL.txt) en tu propia máquina usando la interfaz gráfica.


In [17]:
# Esta celda corre la herramienta FastQC con el archivo sub1_1000.fastq. Usar una sola vez por sesión.
! fastqc sub1_10000.fastq

Started analysis of sub1_10000.fastq
Approx 10% complete for sub1_10000.fastq
Approx 20% complete for sub1_10000.fastq
Approx 30% complete for sub1_10000.fastq
Approx 40% complete for sub1_10000.fastq
Approx 50% complete for sub1_10000.fastq
Approx 60% complete for sub1_10000.fastq
Approx 70% complete for sub1_10000.fastq
Approx 80% complete for sub1_10000.fastq
Approx 90% complete for sub1_10000.fastq
Approx 100% complete for sub1_10000.fastq
Analysis complete for sub1_10000.fastq


FastQC genera dos archivos, uno .html y otro .zip. <br>
Nos enfocaremos en el archivo **.html** ya que se trata de un reporte donde se resumen los resultados. En el archivo .zip también encontramos los resultados representados visualmente pero por separado, además de los datos generados para crear las figuras.

In [18]:
# Esta celda muestra los archivos que manejamos en la nube hasta ahora, durante esta sesión.
! ls

Miniconda3-4.5.12-Linux-x86_64.sh  sub1_10000_fastqc.html
sample_data			   sub1_10000_fastqc.zip
sub1_10000.fastq


A continuación debes descargar el archivo **sub1_10000_fastqc.html** en tu propia máquina. Puedes hacerlo en el menú izquierdo seleccionando la pestaña Files. Encontrarás una lista con los archivos y carpetas disponibles. La opción *Download* está disponible en el menú de tres puntos verticales al mover el cursor sobre el nombre del archivo.<br>
Una vez que hayas descargado sub1_10000_fastqc.html, dale doble click para visualizarlo en tu navegador web. <br>
<br>
En los siguientes segmentos se muestran imagenes del reporte, pero lo ideal es que primero lo descargues y revises en tu navegador web. <br>
<br>
**Estadísticas básicas**:

In [19]:
# Correr esta celda muestra las estadísticas básicas:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/basic_statistics.png',
      width = 704, height = 476)

El resumen anterior incluye características relevantes de las lecturas, como el método de codificación usado para representar información de calidad, en este caso se trata de Sanger (Phred+33). Además entrega el número total de lecturas, en este caso 10000, la longuitud de lecturas, así como el % de GC.<br>
<br>
**Calidad de secuencia por base**:

In [20]:
# Correr esta celda muestra la calidad de secuencia por base:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/per_base_sequence_quality.png',
      width = 1089, height = 635)

El gráfico anterior representa el rango de valores de calidad, en cada posición a lo largo de las lecturas. <br>
El **eje y** muestra los valores de calidad. El fondo del gráfico divide el eje y en tres regiones. <font color='green'>**Verde**</font> = muy alta calidad, <font color='orange'>**Naranjo**</font> = calidad razonable, y <font color='red'>**Rojo**</font> = Baja calidad.<br>
<br>
El **eje x** resumen las posiciones de todas las lecturas. Ten en cuenta que este eje no es uniforme, empieza representando las primeras 10 posiciones de manera individual, pero después agrupa posiciones en ventanas. El rango de posiciones agrupadas va a depender del largo de las lecturas. En este caso las lecturas tienen 250 bases de largo, y las ventanas incluyen 5 bases. El eje x no informa todas las ventanas con bases agrupadas ya que no entrarían. Por ejemplo, la columna inmediatamente a al derecha de la posición **9** no indica que posiciones agrupa, pero entendemos que se trata de las posiciones 10-11-12-13-14 ya que la siguiente columna incluye las posiciones 15-16-17-18-19 (resumido como 15-19).<br>
<br>
En cada columna encontramos un gráfico tipo **caja y bigotes**, que contiene los siguientes elementos:


*   **Mediana**, corresponde a la línea roja central
*   **Rango intercuartil**, representado por una caja amarilla
*   **Percentiles 90 y 10**, visualizados como bigotes
*   **Media**, mostrada como línea azul<br>

¿Por qué los valores de calidad tienden a disminuir hacia el final de las lecturas Illumina? <br>
<br>
Los métodos de secuenciación por síntesis, como Illumina y Sanger, dependen de la enzima polimerasa para incorporar nuevos nucleótidos. Esto es acompañado de etapas de interrupción de cadena con nucleótidos marcados que luego permiten identificar a qué base corresponde.<br>
En el caso de Illumina, esta interrupción es reversible. En cada ciclo se incorpora un nucleótido interrumpiendo la cadena, se registra a qué base corresponde mediante el color del fluoróforo incorporado, para luego liberar dicho fluoróforo, lo que permite que la misma cadena de ADN pueda ser alargada en el siguiente ciclo de síntesis.<br>
Debemos recordar que este proceso implica un conjunto de secuencias que corresponden a copias de sí mismas, es decir, un clúster. Al momento de identificar una base, la detección por color contempla la señal de todo el clúster como un conjunto. Esto debido a que la señal emitida por una sola molécula de fluoróforo en una cadena de ADN no es suficiente para ser registrada por las cámaras del equipo. <br>
<br>
Las lecturas Illumina están limitadas en longuitud debido a que a medida que progresa la síntesis de cadenas de ADN, se van acumulando [errores](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2978646/) en la etapa de interrupción de cadena en cada ciclo. Estos errores son:
*   Interrupción irreversible de cadena (error en el clivaje del fluoróforo)
*   Interrupción deficiente de cadena (incorporación de nucleótido fuera de ciclo)<br>

Este tipo de errores genera, luego de alcanzar cierto largo en las cadenas de ADN, ruido en la señal. Con ruido nos referimos a una señal de color de fluoróforo ambigua, donde es díficil determinar a qué color corresponde exactamente un ciclo en particular. Esta señal ambigua se debe a que ahora el clúster incluye varias cadenas de ADN que durante el mismo ciclo de secuenciación han incorporado fluoróforos de colores diferentes.


¿Qué podemos hacer para mejorar la calidad de nuestros datos? <br>
<br>
Existen herramientas que permiten filtrar y cortar lecturas, según sus valores de calidad. En este caso usaremos Trimmomatic, pero antes de ello seguiremos viendo el reporte de calidad de FastQC. <br>
<br>
**Puntuaciones de calidad por secuencia**.

In [21]:
# Correr esta celda muestra las puntuaciones de calidad por secuencia:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/per_sequence_quality_scores.png',
      width = 1089, height = 635)

La gráfica anterior representa la distribución de calidad promedio. <br>
El **eje y** muestra los valores de frecuencia, mientras que el **eje x** corresponde a valores de calidad promedio por lectura. <br>
Esta sección del reporte nos permite evaluar en torno a qué valores de calidad se agrupan la mayoría de las lecturas.  En este caso vemos que la cantidad de lecturas con un valor de calidad promedio < 20 es mínimo. También destacan dos grupos de lecturas. Uno con valores de calidad entre 21-36, donde se encuentran la mayoría de las lecturas, y otro con valores sobre 36, que incluye aproximadamente 2.200 lecturas. <br>
En este caso los resultados son aceptables, ya que la mayoría de las lecturas tienen valores de calidad promedio sobre 20. En el caso contrario, una moda de valores aparecería alrededor de la columna 20.

**Contenido de secuencia por base** <br>


In [22]:
# Correr esta celda muestra el contenido de secuencia por base:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/per_base_sequence_content.png',
      width = 1089, height = 635)

Este segmento del reporte destaca el contenido de secuencias, por cada base. <br>
El **eje y** muestra el contendio en porcentaje. El **eje x** corresponde a las posiciones de todas las lecturas, de manera similar a lo visto en la sección "calidad de secuencia por base". <br>
En cada posición de las lecturas se resume el contenido de <font color='green'>**Adenina**</font>, <font color='black'>**Guanina**</font>, <font color='blue'>**Citosina**</font>, y <font color='red'>**Timina**</font> en colores. <br>
Si bien el valor esperado del contenido de <font color='green'>**A**</font>, <font color='black'>**G**</font>, <font color='blue'>**C**</font>, o <font color='red'>**T**</font> debería ubicarse alrededor del 25%, estos valores pueden variar a través del genoma y también entre especies. <br>
En este caso vemos que los porcentajes de contenido cambian notoriamente en las primeras 10 bases. A partir de ahí, los valores en general se mantienen por el resto de las lecturas sin modificaciones evidentes. <br>
<br>
¿Por qué las primeras 10 bases tienen valores tan alejados de lo esperado? ¿Por qué varían tanto? <br>
<br>
Lo visto en las primeras 10 bases corresponde a un sesgo técnico común de librerías de secuenciación que utilizan la tecnología de *Tagmentation*. Consiste en [fragmentación y etiquetado simultáneo](https://www.nature.com/articles/nmeth.f.272) de moléculas de ADN durante la preparación de muestras. Su uso tiene claras ventajas al reducir tiempo y cantidad de recursos necesarios en cada ronda de secuenciación, pero se ha observado que [tienden a alterar los valores de contenido de secuencia por base](https://www.frontiersin.org/articles/10.3389/fpubh.2019.00241/full) en las primeras 10-15 bases en lecturas. Esto debido a que el procedimiento incorpora etiquetas en el extremo 5' de los fragmentos. Estas etiquetas consisten en moléculas de ADN cortas, y por ende pueden enriquecer los valores de contenido de secuencia. 


**Contenido de GC por secuencia** <br>

In [ ]:
# Correr esta celda muestra el contenido de GC por secuencia:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/per_sequence_GC_content.png',
      width = 1089, height = 635)

Esta gráfica muestra la distribución del **contenido de GC** en las lecturas, y la compara con una distribución normal teórica. El **eje y** indica valores de frecuencia, mientras que el **eje x** corresponde al contenido de GC promedio.<br>
En una hebra de ácido nucléico como ADN o ARN, el contenido de GC corresponde al porcentaje de nucleótidos que contienen <font color='black'>**Guanina**</font> o <font color='blue'>**Citosina**</font> como base nitrogenada. <br>
En una molécula de ADN <font color='black'>**G**</font> y <font color='blue'>**C**</font> son complementarias, por ello el contenido de GC es el mismo en ambas hebras. El contenido de GC varia a lo largo del genoma, y también entre especies. [En humanos](https://www.nature.com/articles/35057062) el promedio a lo largo del genoma es de 41%, y puede variar entre 33.1% y 59.3% en regiones menores a 300kb. <br>
Una distribución del contenido GC con una forma inusual puede indicar contaminación.

**Contenido de N por base**

In [ ]:
# Correr esta celda muestra el contenido de N por base:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/per_base_N_content.png',
      width = 1089, height = 635)

En ocasiones los secuenciadores reciben señales ambiguas. Esto los limita al momento de interpretar la señal en una base determinada. <br>
En equipos Illumina estas señales corresponden a emisiones de fluorescencia, mientras que en equipos Nanopore las señales obtenidas son corrientes de iones. <br> 
Cuando no existe certeza en la interpretación de una señal los secuenciadores entregan una N como resultado indicando que esa base en particular es desconocida. <br>
Este segmento del reporte resume el **contenido de N** en las lecturas. El **eje y** muestra el contenido en porcentaje, y el **eje x** indica la posición en las lecturas.<br>
La presencia de N en las lecturas indica pérdida de calidad y su interpretación se complementa con segmentos anteriores.

**Distribución de longitud de secuencia**

In [ ]:
# Correr esta celda muestra la distribución de longitud de secuencia:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/sequence_length_distribution.png',
      width = 1089, height = 635)

Esta gráfica muestra la distribución del tamaño de lecturas. <br>
El tamaño de lecturas puede ser uniforme o variado según el tipo de tecnología de secuenciación. En este caso podemos ver que todas las lecturas tienen una longitud de 250 pares de bases. <br>
Como mencionamos en la sección de calidad de secuencia por base, en tecnologías como Illumina la calidad tiende a disminuir hacia el final de la secuencia. Una medida para filtrar lecturas consiste en recortar los extremos que presentan menor calidad. De esta manera se obtienen lecturas de longitud variada. Sin embargo, esto no representa limitaciones para análisis posteriores.

**Niveles de duplicación de secuencia** <br>

In [ ]:
# Correr esta celda muestra los niveles de duplicación de secuencia:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/sequence_duplication_levels.png',
      width = 1089, height = 635)

Este segmento del reporte representa los niveles de duplicación de secuencias. <br>
Por duplicación nos referimos a una secuencia que se repite dos o más veces. Los niveles de duplicación se muestran en el **eje x** y a partir de 10 repeticiones en adelante se agrupan en órdenes de magnitud cada vez mayores. Los porcentajes de secuencias que corresponden a los distintos niveles de duplicación se destacan en el **eje y**.<br>
Este análisis entrega dos curvas, el <font color='blue'>**% de secuencias totales**</font>, y el <font color='red'>**% de secuencias deduplicadas**</font>. Las secuencias totales incluyen a todos los tipos de secuencias y a sus respectivas copias. Por ejemplo si las secuencias *A*, *B*, y *C* se repiten 2, 5, y 10 veces respectivamente, la curva de <font color='blue'>**% de secuencias totales**</font> considerará los tres tipos de secuencias (A, B, y C) y sus respectivas copias, en total 17 secuencias. Por otro lado el <font color='red'>**% de secuencias deduplicadas**</font> sólo considera los tipos de secuencias, en este caso 3 (A, B, y C). <br>
El porcentaje de secuencias que se mantienen luego de la deduplicación de datos se indica en el encabezado del gráfico. <br>
Idealmente, las librerías de secuenciación contienen secuencias diversas y sus niveles de duplicación se mantienen en el extremo izquierdo de la gráfica. Sin embargo, la presencia de contaminación puede verse reflejada en este análisis como modas en el extremo derecho de la gráfica. <br>
Además, las librerías de ARN suelen tener mayores niveles de duplicación debido a que naturalmente varios transcritos se expresan en mayor proporción que el resto de transcritos. En dichos casos se espera que este módulo entregue señales de advertencia.

**Secuencias enriquecidas y contenido de adaptadores**

In [1]:
# Correr esta celda muestra la presencia de secuencias enriquecidas y el contenido de adaptadores:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/overrepresented_sequences_and_adapter_content.png',
      width = 1089, height = 635)

Los últimos dos segmentos del reporte destacan la presencia de secuencias enriquecidas y adaptadores, respectivamente. <br>
Una librería de secuenciación de ADN genómico debería ser lo más diversa posible, y ninguna secuencia en particular debiese verse enriquecida. En caso contrario, el segmento de secuencias enriquecidas destacará una tabla con secuencias que sobrepasan el 0.1% del total.<br>
<br>
En la gráfica del contenido de adaptadores, el **eje x** muestra la posición de la lectura en la cual se ha identificado la presencia de adpatadores. El **eje y** corresponde al porcentaje de lecturas que contiene adaptadores en una determinada posición. La curva es acumulativa, y se observa que los adaptadores tienen a incorporarse en el extremo 3' de las lecturas, ¿Por qué?<br>
<br>
La preparación de librerías tiene limitaciones técnicas. Si bien se busca extraer un tamaño determinado de fragmentos de ADN a partir de la muestra, es común que se termine extrayendo una distribución de tamaños, más que un solo tamaño en particular. <br>
Las librerías de secuenciación consisten en fragmentos de ADN de secuencia desconocida obtenidos a partir de la muestra, rodeados por moléculas cortas de ADN artificial de secuencia conocida llamados adaptadores. <br>
En tecnologías Illumina los [adaptadores](https://support.illumina.com/bulletins/2020/06/illumina-adapter-portfolio.html) cumplen la función de permitir la unión de las secuencias a la celda de flujo para formar clusters, así como diferenciar las hebras de ADN. <br>
Durante la secuenciación, el número de ciclos de síntesis determina el largo de la lectura obtenida. Como mencionamos antes, existen errores que limitan esta longitud. Idealmente en una molécula los ciclos de secuenciación abarcan únicamente al fragmento de ADN obtenido a partir de la muestra. Dicho fragmento tiene secuencia desconocida y es de interés para nosotros. Pero en ocasiones, si el fragmento obtenido desde la muestra tiene un tamaño menor al número de ciclos de secuenciación, la lectura incorporará la secuencia del adaptador, la cual no es de interés. <br>
Esta incorporación de adaptadores ocurre en el extremo 3' de la lectura, y el número de bases incorporado corresponde a la longitud faltante del fragmento de muestra para completar los cíclos de secuenciación.<br>
Afortunadamente es posible corregir esta limitación con herramientas que identifican adaptadores y los remueven de nuestras lecturas.<br>
La incorporación de adaptadores se resumen en el siguiente esquema realizado por [ecSeq Bioinformatics](https://www.ecseq.com/support/ngs/trimming-adapter-sequences-is-it-necessary).

In [4]:
# Correr esta celda muestra la incorporación de adaptadores en lecturas:
from IPython.display import Image
Image(url='https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/adapters_ecseq.png',
      width = 450, height = 350)

### Filtro de datos por calidad y recorte de adaptadores


Ahora que ya identificamos problemas de calidad y artefactos en las lecturas, podemos continuar con el segundo paso del control de calidad, filtrado y recorte. <br>
<br>
Pero antes debes responder la primera pregunta de este práctico:<br>
0.- ¿Qué segmentos del reporte obtenido por FastQC presentaron problemas?

In [ ]:
# Escribe tu respuesta entre las comillas triples, de esta manera: """ Tu respuesta """
"""
Reemplaza este texto con tu respuesta.

"""

Existen varias herramientas que permiten filtrar y recortar lecturas, en este caso usaremos [Trimmomatic](http://www.usadellab.org/cms/?page=trimmomatic). <br>
Trimmomatic incluye varias funciones, entre ellas: <br>
1. Buscar y recortar secuencias de adaptadores en el extremo 3' de las lecturas. En este caso las secuencias de adaptadores se especifican en el archivo **NexteraPE-PE.fa**.
2. Recortar un número determinado de bases al inicio de las lecturas, independiente a su calidad. En este caso recortaremos 12 bases.
3. Recortar bases que posean un valor de calidad menor al especificado. Este recorte ocurre desde el extremo 3' de las lecturas. En este caso el mínimo de calidad aceptado será 25.
4. Eliminar lecturas que posean una longitud menor a la deseada. En este caso 36 bp.<br>


A continuación descargaremos el archivo **NexteraPE-PE.fa** en la nube de Google Colab. 

In [28]:
! wget https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/NexteraPE-PE.fa

--2021-08-04 16:21:49--  https://raw.githubusercontent.com/mmontes-umag/datos-intro-bioinf/main/Control_de_calidad/NexteraPE-PE.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 239 [text/plain]
Saving to: ‘NexteraPE-PE.fa’

NexteraPE-PE.fa     100%[===================>]     239  --.-KB/s    in 0s      

2021-08-04 16:21:49 (7.58 MB/s) - ‘NexteraPE-PE.fa’ saved [239/239]



<font color='red'>**Atención:**</font> es importante que revises si el archivo **NexteraPE-PE.fa** se descargó correctamente. En caso contrario los resultados obtenidos con Trimmomatic se verán afectados negativamente y ello perjudicará tus respuestas al final de este práctico. <br>
Para asegurarte que cuentas con el archivo corre la siguiente celda y busca por su nombre: **NexteraPE-PE.fa**. <br>
Si el archivo está presente, corre la celda de código subsiguiente, en la que usamos Trimmomatic, antes de que se reinicie la sesión de Google Colab.

In [33]:
# Esta celda muestra los archivos que manejamos en la nube hasta ahora, durante esta sesión.
! ls

Miniconda3-4.5.12-Linux-x86_64.sh  sub1_10000_fastqc.zip
NexteraPE-PE.fa			   sub1_10000Trim.fastq
sample_data			   sub1_10000Trim_fastqc.html
sub1_10000.fastq		   sub1_10000Trim_fastqc.zip
sub1_10000_fastqc.html


En la siguiente celda usamos Trimmomatic con las cuatro funciones mencionadas. El resultado consiste en el archivo **sub1_10000Trim.fastq**.

In [29]:
! trimmomatic SE sub1_10000.fastq sub1_10000Trim.fastq ILLUMINACLIP:NexteraPE-PE.fa:2:30:10 HEADCROP:12 TRAILING:25 MINLEN:36

TrimmomaticSE: Started with arguments:
 sub1_10000.fastq sub1_10000Trim.fastq ILLUMINACLIP:NexteraPE-PE.fa:2:30:10 HEADCROP:12 TRAILING:25 MINLEN:36
Automatically using 2 threads
Using PrefixPair: 'AGATGTGTATAAGAGACAG' and 'AGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'GTCTCGTGGGCTCGGAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'TCGTCGGCAGCGTCAGATGTGTATAAGAGACAG'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTCCGAGCCCACGAGAC'
Using Long Clipping Sequence: 'CTGTCTCTTATACACATCTGACGCTGCCGACGA'
ILLUMINACLIP: Using 1 prefix pairs, 4 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Reads: 10000 Surviving: 9871 (98.71%) Dropped: 129 (1.29%)
TrimmomaticSE: Completed successfully


<font color='red'>**Atención:**</font> Si al correr una celda de código te encuentras con mensajes de error como "command not found", "no such file or directory", o "skipping -nombre archivo- which didn't exist, or couldn't be read", eso quiere decir que la sesión de Google Colab se reinició. Para solucionarlo basta con volver a correr únicamente las celdas que corresponden a **instalación de herramientas** y **descarga de datos** en orden.<br>
Una vez hecho esto, vuelve a correr la celda que entregaba el mensaje de error.

In [30]:
# Esta celda muestra los archivos que manejamos en la nube hasta ahora, durante esta sesión.
! ls

Miniconda3-4.5.12-Linux-x86_64.sh  sub1_10000.fastq	   sub1_10000Trim.fastq
NexteraPE-PE.fa			   sub1_10000_fastqc.html
sample_data			   sub1_10000_fastqc.zip


<font color='red'>**Atención:**</font> si en la celda anterior se incluyen los archivos en formato .fastq, .html y .zip, pero **no** se incluye el archivo **NexteraPE-PE.fa**, deberás correr el análisis nuevamente pero esta vez asegurandote que has descargado NexteraPE-PE.fa.

Como vemos, ya obtuvimos el archivo de lecturas filtrado y recortado por Trimmomatic, **sub1_10000Trim.fastq**. A continuación evaluaremos la calidad de las lecturas obtenidas con FastQC:

In [31]:
! fastqc sub1_10000Trim.fastq

Started analysis of sub1_10000Trim.fastq
Approx 10% complete for sub1_10000Trim.fastq
Approx 20% complete for sub1_10000Trim.fastq
Approx 30% complete for sub1_10000Trim.fastq
Approx 40% complete for sub1_10000Trim.fastq
Approx 50% complete for sub1_10000Trim.fastq
Approx 60% complete for sub1_10000Trim.fastq
Approx 70% complete for sub1_10000Trim.fastq
Approx 80% complete for sub1_10000Trim.fastq
Approx 90% complete for sub1_10000Trim.fastq
Analysis complete for sub1_10000Trim.fastq


In [32]:
# Esta celda muestra los archivos que manejamos en la nube hasta ahora, durante esta sesión.
! ls

Miniconda3-4.5.12-Linux-x86_64.sh  sub1_10000_fastqc.zip
NexteraPE-PE.fa			   sub1_10000Trim.fastq
sample_data			   sub1_10000Trim_fastqc.html
sub1_10000.fastq		   sub1_10000Trim_fastqc.zip
sub1_10000_fastqc.html


Como esperábamos, la herramienta FastQC nos entregó un reporte en formato .html. <br>
En esta ocasión deberás descargar el reporte .html y revisarlo en tu propio navegador para resolver las siguientes preguntas. <br>
Recuerda que puedes hacerlo en el menú izquierdo seleccionando la pestaña Files. Encontrarás una lista con los archivos y carpetas disponibles. La opción *Download* está disponible en el menú de tres puntos verticales al mover el cursor sobre el nombre del archivo.<br>
Una vez que hayas descargado **sub1_10000Trim_fastqc.html**, dale doble click para visualizarlo en tu navegador web. <br>

### Preguntas finales
A continuación debes comparar ambos reportes obtenidos por FastQC:
1. **sub1_10000_fastqc.html**, obtenido directamente de las lecturas sin filtrar o recortar.
2. **sub1_10000Trim_fastqc.html**, obtenido luego de filtrar por calidad y recortar adaptadores. 

1.- ¿En qué se diferencian las estadísticas básicas de ambos reportes?

In [ ]:
# Escribe tu respuesta entre las comillas triples, de esta manera: """ Tu respuesta """
"""
Reemplaza este texto con tu respuesta.

"""

2.- ¿Cuál de las cuatro funciones mencionadas de Trimmomatic tuvo un efecto en la calidad de secuencias por base?

In [1]:
# Escribe tu respuesta entre las comillas triples, de esta manera: """ Tu respuesta """
"""
Reemplaza este texto con tu respuesta.

"""

'\nReemplaza este texto con tu respuesta.\n\n'

3.- Respecto a las puntuaciones de calidad por secuencia, en el archivo obtenido luego de filtrar y recortar, ¿cuál es el valor de calidad promedio donde se encuentra la mayoría de las lecturas?

In [ ]:
# Escribe tu respuesta entre las comillas triples, de esta manera: """ Tu respuesta """
"""
Reemplaza este texto con tu respuesta.

"""

4.- Respecto a las puntuaciones de calidad por secuencia, al comparar ambos reportes, en qué se diferencian ambas curvas? <br>
(máximo 50 palabras)

In [ ]:
# Escribe tu respuesta entre las comillas triples, de esta manera: """ Tu respuesta """
"""
Reemplaza este texto con tu respuesta.

"""

5.- Cuál de las cuatro funciones mencionadas de Trimmomatic tuvo un efecto en el contenido de secuencia por base?

In [ ]:
# Escribe tu respuesta entre las comillas triples, de esta manera: """ Tu respuesta """
"""
Reemplaza este texto con tu respuesta.

"""

6.- Respecto al contenido de adaptadores, cuál es la diferencia entre ambos reportes?

In [ ]:
# Escribe tu respuesta entre las comillas triples, de esta manera: """ Tu respuesta """
"""
Reemplaza este texto con tu respuesta.

"""

Más acerca de [FastQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/). <br>
¿Qué pasa si tengo múltiples archivos y quiero resumir mis resultados FastQC en un solo reporte? Recomendamos revisar la herramienta [MultiQC](https://multiqc.info/).